In [1]:
from torch.utils.data import DataLoader, Dataset
from tokenizers import SentencePieceBPETokenizer
import json
import pandas as pd
import numpy as np
import logging
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import math
import os
from argparse import ArgumentParser
import torch
from tokenizers import SentencePieceBPETokenizer
from torch.optim import Adam
from tqdm import tqdm_notebook
from transformers import GPT2LMHeadModel, get_linear_schedule_with_warmup
from korquad_qg.config import QGConfig
from korquad_qg.utils import TqdmLoggingHandler
from typing import List, NamedTuple, Optional, Tuple

# 코쿼드 데이터셋을 이용한 GPT2 데이터셋 생성

In [ ]:
with open('korquad.json', 'r') as f:

    json_data = json.load(f)

context_list = []
answers_list = []
question_list = []

for data in json_data['data']:
    for sub_data in data['paragraphs']:
        context = sub_data['context']
        for qa in sub_data['qas']:
            context_list.append(context)
            answers_list.append(qa['answers'][0]['text'])
            question_list.append(qa['question'])
            if len(qa['answers']) > 1:
                print(qa['answers'])
                
data = {
        'context':context_list,
        'answers': answers_list,
        'question':question_list
    }
df = pd.DataFrame(data)
df.to_csv('KorQuad_train_V1.csv',index=False)

# 토크나이저 생성

In [ ]:
txt_f = open("sentences.txt", 'r')
all_dict = []
max_len = 0
count = 0
for line in tqdm_notebook(txt_f.readlines()):
    if '\n' in line:
        line = line.replace('\n', '')
    if '##' in line:
        line = line.replace('##', '')
    if len(line) > max_len:
        max_len = len(line)
    all_dict.append(line)
    count += 1
print(max_len)


f = open("data_save_test.txt", 'w')
for i in all_dict:
    f.write(i+'\n')
f.close()

# Initialize a tokenizer
tokenizer = SentencePieceBPETokenizer()
# Then train it!

tokenizer.train_from_iterator([all_dict], vocab_size=300000, min_frequency=1, limit_alphabet=100000)
tokenizer.save("./history_tokenizer.json",pretty=True)
tokenizer.save_model(directory='./')

In [ ]:
txt_f = open("sentences.txt", 'r')
max_len = 512
count = 0
tok = PreTrainedTokenizerFast(tokenizer_file='Tokenizer/History/history_tokenizer.json')
for line in tqdm_notebook(txt_f.readlines()):
    if '\n' in line:
        line = line.replace('\n', '')
    if '##' in line:
        line = line.replace('##', '')
    if len(tok.tokenize(line)) + 7 > max_len:
        max_len = len(line)
        count += 1
print(max_len, count)

# 데이터셋 생성

In [2]:
Q_TKN = '<q>'
A_TKN = '<a>'
BOS = '<s>'
EOS = '</s>'
MASK = '<mask>'
C_TKN = '<c>'
PAD = '<pad>'
TOKENIZER = PreTrainedTokenizerFast(tokenizer_file='Tokenizer/tokenizer.json',
            bos_token=BOS, eos_token=EOS, unk_token='<unk>', 
            pad_token=PAD, mask_token=MASK)

## 데이터 셋 클래스

In [3]:
GPTDecodingInputType = Tuple[torch.Tensor, torch.Tensor]
GPTInputsType = Tuple[torch.Tensor, torch.Tensor, torch.Tensor]
GPTFeaturesType = Tuple[List[int], List[float], List[int]]

class QAExample(NamedTuple):
    context: str
    answer: str
    question: Optional[str] = None

def load_korquad_dataset(dataset_path: str) -> List[QAExample]:
    korquad = [pd.read_csv(dataset_path)]
    max_len = 512
    examples = []
    for document in korquad:
        for i in tqdm_notebook(range(len(document))):
            if len(TOKENIZER.tokenize(document["context"][i])) + 10 <= max_len:
                example = QAExample(document["context"][i], document["answers"][i], document["question"][i])
                examples.append(example)
        
    return examples
    
def dynamic_padding_collate_fn(features: List[GPTFeaturesType]) -> GPTInputsType:
    max_seq_len = max([len(feature[0]) for feature in features])
    input_ids, attention_mask, labels = [], [], []

    for feature in features:
        padded_input_ids = feature[0] + [0] * (max_seq_len - len(feature[0]))
        padded_attention_mask = feature[1] + [0.0] * (max_seq_len - len(feature[1]))
        padded_labels = feature[2] + [-100] * (max_seq_len - len(feature[2]))

        input_ids.append(padded_input_ids)
        attention_mask.append(padded_attention_mask)
        labels.append(padded_labels)

    return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(labels)

In [4]:
class HistoryQGDataset(Dataset):
    def __init__(
        self,
        examples: List[QAExample],
        tokenizer: SentencePieceBPETokenizer,
        max_sequence_length: int,
        is_train: bool = True,
    ) -> None:
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length

        self.sos_token = tokenizer.convert_tokens_to_ids("<s>")
        self.eos_token = tokenizer.convert_tokens_to_ids("</s>")
        self.question_prefix_tokens = tokenizer.convert_tokens_to_ids('<q>')

        self.is_train = is_train

    def __getitem__(self, index: int) -> GPTFeaturesType:
        example = self.examples[index]

        context_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(f"<c>{example.context}"))
        answer_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(f"<a>{example.answer}"))
        question_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(f"{example.question}"))
        
        # [SOS] + 문맥:CONTEXT + 정답:ANSWER + 질문:
        conditional_tokens_len = 1 + len(context_tokens) + len(answer_tokens) + 1
        # QUESTION + [EOS]
        post_tokens_len = len(question_tokens) + 1

        if conditional_tokens_len + post_tokens_len > self.max_sequence_length:
            available_seq_len = (
                self.max_sequence_length - conditional_tokens_len - post_tokens_len + len(context_tokens)
            )
            context_tokens = context_tokens[:available_seq_len]

        conditional_tokens = [self.sos_token] + context_tokens + answer_tokens + [self.question_prefix_tokens]
        post_tokens = question_tokens + [self.eos_token]
        input_ids = conditional_tokens + post_tokens

        labels = input_ids if self.is_train else ([-100] * len(conditional_tokens)) + post_tokens
        attention_mask = [1.0] * len(input_ids)

        assert len(input_ids) <= self.max_sequence_length

        return input_ids, attention_mask, labels
        #return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(labels)

    def __len__(self) -> int:
        return len(self.examples)


In [5]:
def _create_logger(output_dir: str):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(asctime)s] %(message)s")

    file_handler = logging.FileHandler(os.path.join(output_dir, "train.log"))
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    handler = TqdmLoggingHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s"))
    logger.addHandler(handler)
    return logger

In [6]:
def _validate(
    model: GPT2LMHeadModel,
    dev_dataloader: DataLoader,
    device: torch.device,
    logger: logging.Logger,
    global_step: int,
):
    model.eval()
    loss_list = []
    for batch_data in tqdm_notebook(dev_dataloader, desc="[EVAL]"):
        with torch.no_grad():
            input_ids, attention_mask, labels = tuple(value.to(device) for value in batch_data)
            model_outputs = model.forward(input_ids, attention_mask=attention_mask, labels=labels, return_dict=True)
            loss_list.append(model_outputs.loss.item())

    mean_loss = np.mean(loss_list)
    logger.info(f"[EVAL] global_step:{global_step} loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}")
    model.train()

# 모델 학습

In [7]:
config = QGConfig()

tokenizer = PreTrainedTokenizerFast(tokenizer_file='Tokenizer/tokenizer.json',
            bos_token=BOS, eos_token=EOS, unk_token='<unk>', 
            pad_token=PAD, mask_token=MASK)

logger = _create_logger(output_dir=config.output_dir)
logger.info("============================")
for key, value in config._asdict().items():
    logger.info(f"{key:30}:{value}")
logger.info("============================")
torch.manual_seed(config.random_seed)

logger.info("loading train dataset")
train_examples = load_korquad_dataset(config.train_dataset)
train_dataset = HistoryQGDataset(train_examples, tokenizer, config.max_sequence_length)
train_dataloader = torch.utils.data.DataLoader(train_dataset, 16, shuffle=True, collate_fn=dynamic_padding_collate_fn)
#train_dataloader = torch.utils.data.DataLoader(train_dataset, 1, shuffle=True)
logger.info("loading dev dataset")
dev_examples = load_korquad_dataset(config.dev_dataset)
dev_dataset = HistoryQGDataset(dev_examples, tokenizer, config.max_sequence_length, is_train=False)
dev_dataloader = DataLoader(dev_dataset, 16, collate_fn=dynamic_padding_collate_fn)
#dev_dataloader = DataLoader(dev_dataset, 1)

#model 생성
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.load_state_dict(torch.load('outputs/After_History_Finetuning/gpt2_step_18000.pth'))
#model.load_state_dict(torch.load('outputs/After_History_Finetuning/gpt2_step_18000.pth', map_location="cpu"))
print(model.transformer.wte.weight.shape[0], len(tokenizer.vocab))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = Adam(model.parameters(), lr=config.lr)
total_steps = len(train_dataloader) * config.epochs
warmup_steps = int(total_steps * config.warmup_ratio)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

loss_list_between_log_interval = []
for epoch_id in range(config.epochs):
    for step_index, batch_data in tqdm_notebook(
            enumerate(train_dataloader), f"[TRAIN] EP:{epoch_id}", total=len(train_dataloader)
    ):
        global_step = len(train_dataloader) * epoch_id + step_index + 1
        optimizer.zero_grad()

        token_ids, attention_mask, labels = tuple(value.to(device) for value in batch_data)
        model_outputs = model.forward(token_ids, attention_mask=attention_mask, labels=labels, return_dict=True)
        model_outputs.loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
        optimizer.step()
        scheduler.step()

        # for logging
        loss_list_between_log_interval.append(model_outputs.loss.item())

        if global_step % config.train_log_interval == 0:
            mean_loss = np.mean(loss_list_between_log_interval)
            logger.info(
                    f"EP:{epoch_id} global_step:{global_step} "
                    f"loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}"
                )
            loss_list_between_log_interval.clear()
            
        if global_step % config.validation_interval == 0:
                _validate(model, dev_dataloader, device, logger, global_step)
                
        if global_step % config.save_interval == 0:
            state_dict = model.state_dict()
            model_path = os.path.join(config.output_dir, f"gpt2_step_{global_step}.pth")
            torch.save(state_dict, model_path)

2021-11-14 12:48:42,438 - ============================
2021-11-14 12:48:42,440 - train_dataset                 :data/History_train_After_FT_V3.csv
2021-11-14 12:48:42,440 - dev_dataset                   :data/History_train_After_FT_V1.csv
2021-11-14 12:48:42,441 - max_sequence_length           :512
2021-11-14 12:48:42,441 - epochs                        :100
2021-11-14 12:48:42,441 - lr                            :5e-05
2021-11-14 12:48:42,441 - train_batch_size              :2
2021-11-14 12:48:42,441 - dev_batch_size                :2
2021-11-14 12:48:42,442 - output_dir                    :outputs/
2021-11-14 12:48:42,442 - grad_clip                     :1.0
2021-11-14 12:48:42,442 - warmup_ratio                  :0.1
2021-11-14 12:48:42,442 - train_log_interval            :100
2021-11-14 12:48:42,442 - validation_interval           :1000
2021-11-14 12:48:42,442 - save_interval                 :1000
2021-11-14 12:48:42,442 - random_seed                   :0
2021-11-14 12:48:42,443 - 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



2021-11-14 12:48:46,129 - loading dev dataset



51200 51200


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...


2021-11-14 15:32:33,088 - [EVAL] global_step:19000 loss:0.0590 perplexity:1.0608
2021-11-14 15:33:19,839 - EP:32 global_step:19100 loss:0.1783 perplexity:1.1952
2021-11-14 15:34:05,802 - EP:32 global_step:19200 loss:0.1848 perplexity:1.2029
2021-11-14 15:34:51,922 - EP:32 global_step:19300 loss:0.1869 perplexity:1.2055
2021-11-14 15:35:38,473 - EP:32 global_step:19400 loss:0.1896 perplexity:1.2087
2021-11-14 15:36:23,750 - EP:32 global_step:19500 loss:0.1961 perplexity:1.2167



2021-11-14 15:37:07,844 - EP:33 global_step:19600 loss:0.1895 perplexity:1.2087
2021-11-14 15:37:54,473 - EP:33 global_step:19700 loss:0.1709 perplexity:1.1864
2021-11-14 15:38:40,815 - EP:33 global_step:19800 loss:0.1781 perplexity:1.1949
2021-11-14 15:39:27,382 - EP:33 global_step:19900 loss:0.1814 perplexity:1.1989
2021-11-14 15:40:10,322 - EP:33 global_step:20000 loss:0.1893 perplexity:1.2085



2021-11-14 15:41:08,033 - [EVAL] global_step:20000 loss:0.0584 perplexity:1.0601
2021-11-14 15:41:54,798 - EP:33 global_step:20100 loss:0.1886 perplexity:1.2076



2021-11-14 15:42:39,719 - EP:34 global_step:20200 loss:0.1797 perplexity:1.1969



KeyboardInterrupt: 